# Cleaning downloaded Data from GISAID

Author: Alexander Maksiaev

Purpose: Clean downloaded data from GISAID

In [1]:
# Housekeeping

import os
import glob 
import pandas as pd
import importlib
import utils  
importlib.reload(utils)
from utils import * 

home = "C:/Users/maksi/Documents/Statistics/Projects/Avian_Flu_Files/"
downloads = "C:/Users/maksi/Documents/Statistics/Projects/Avian_Flu_Files/GISAID_Files/H5N1-NorthAmerican-gisaid_epiflu_datasets_jan1-2024_to_mar18-2025_v2/"
# downloads = home + "/GISAID_Files"
fasta_files = home + "/GISAID_Complete_Fasta_Files/"

os.chdir(downloads)



            

In [2]:
# Run through all files -- this should be a loop

all_metadata_files = []
all_fasta_files = []

for dirpath, dirs, files in os.walk(downloads):
    for file in files:
        file_name = os.path.join(dirpath, file)
        if ".xls" in file_name:
            # print(file_name)
            metadata = pd.read_excel(file_name, engine="xlrd")
            all_metadata_files.append(metadata)
        if ".fasta" in file_name:
            fasta_file = fasta_df(file_name)
            all_fasta_files.append(fasta_file)

# print(all_metadata_files[0])

unique_animals_all = []

# print(len(all_fasta_files))

os.chdir(home)

segment_fastas = []
unique_segments = []
for i, fasta in enumerate(all_fasta_files):
    # fasta = fasta_df(f_file)
    metadata = all_metadata_files[i]

    unique_animals = sort_animals(fasta)
    unique_animals_all.append(unique_animals)

    # b313_file, d11_file = separate_b313_d11_xls(metadata, fasta)
    animals_ref = pd.read_csv("animals_ref1.csv")

    fastas, unique_segments = separate_fasta_by_seg(metadata, fasta, animals_ref) #, b313_file, d11_file)

    segment_fastas.append(fastas)

# Flatten unique_animals
every_unique_animal = []
for l in unique_animals_all:
    for animal in l:
        every_unique_animal.append(animal)



c:\Users\maksi\Documents\Statistics\Projects\Avian_Flu\utils.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fasta_seg["Genotype"] = fasta_gen
c:\Users\maksi\Documents\Statistics\Projects\Avian_Flu\utils.py:145: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fasta_seg["New_Name"] = new_name
c:\Users\maksi\Documents\Statistics\Projects\Avian_Flu\utils.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [3]:
print(len(segment_fastas))

print(every_unique_animal)

7
['duck', 'western_gull', 'cat', 'goat', 'lion', 'goose', 'bobcat', 'bottlenose_dolphin', 'pigeon', 'ibis', 'savannah_cat', 'mountain_lion', 'ermine', 'owl', 'swine', 'lynx', 'dairy_cow', 'tiger', 'turkey', 'geoffroys_cat', 'pheasant', 'common_grackle', 'chicken', 'hawk', 'fox', 'environment', 'sanderling', 'serval', 'guineafowl', 'duck', 'cackling_goose', 'barn_owl', 'red-tailed_hawk', 'western_gull', 'tundra_swan', 'cat', 'green-winged_teal', 'goose', 'great_blue_heron', 'bobcat', 'hooded_merganser', 'northern_shoveler', 'black_vulture', 'double-crested_co', 'sandhill_crane', 'red-shouldered_hawk', 'bald_eagle', 'mallard', 'merganser', 'cougar', 'black-crowned_night-heron', 'gadwall', 'american_wigeon', 'ring-billed_gull', 'common_raven', 'trumpeter_swan', 'peafowl', 'dairy_cow', 'falcon', 'coopers_hawk', 'crested_caracara', 'snowy_owl', 'turkey_vulture', 'turkey', 'harbor_seal', 'red-breasted_merganser', 'wood_duck', 'snow_goose', 'canada_goose', 'flamingo', 'crow', 'great_horned_o

In [4]:
# Merge them all together into 16 files -- 8 segments x 2 genotypes

# 16 files needed
huge_fasta = pd.DataFrame()

for fastas in segment_fastas: # 7 batches
    # print(len(fastas))
    # break
    for f in fastas: # 16 files per batch 
        # print(f)
        # break 
        huge_fasta = pd.concat([huge_fasta, f])

# print(huge_fasta.columns)

# Now separate huge_fasta into 16 fastas
big_fastas = []

genotypes = ["B3.13", "D1.1"]
for gen in genotypes:
    big_fasta = huge_fasta[huge_fasta["Genotype"] == gen]
    for seg in unique_segments:
        seg_specific_fasta = big_fasta[big_fasta["Segment"] == seg]
        big_fastas.append(seg_specific_fasta)

# Now that we have 16 fastas, write the files
for fasta in big_fastas:

    # Create a dictionary to create a file
    fasta_df = fasta[["New_Name", "Sequence"]]
    fasta_dict = pd.Series(fasta_df.Sequence.values,index=fasta_df.New_Name).to_dict()

    # Create fasta file 
    output_path = fasta_files + fasta["Genotype"].values[0] + "_" + fasta["Segment"].values[0] + ".fasta" # Genotype and Segment should all be the same
    output_file = open(output_path, "w")
    for item in fasta_dict.keys():
        value = fasta_dict[item] + "\n"
        # print(value)
        output_file.write(item)
        output_file.write(value)
    output_file.close()

In [5]:
# Rename host type

unique_animals_set = list(set(every_unique_animal))
animals_df = pd.DataFrame(columns=["avian", "cattle", "feline", "other_mammal", "human", "other"])
animals_df["other"] = unique_animals_set # to sort

os.chdir(home)
animals_df.to_csv("animals_ref2.csv")